In [1]:
import tensorflow as tf
print(tf.config.list_physical_devices('GPU'))


2025-08-18 18:52:46.267244: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1755535966.291668   59792 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1755535966.301310   59792 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1755535966.320820   59792 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1755535966.320845   59792 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1755535966.320849   59792 computation_placer.cc:177] computation placer alr

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


## Importy

In [2]:
import cv2
import mediapipe as mp
import numpy as np
from tensorflow.keras.models import load_model


## Wczytanie modelu

In [19]:
models = {
    "lstm_2025_08_18": load_model('../model/2025-08-18/lstm.keras'),
    # "cnn_1d_2025_07_14": load_model('../model/2025-07-14/cnn_1d_model.keras'),
}


## Wczytanie etykiet z pliku tekstowego


In [4]:
with open('../model/2025-07-14//labels.txt', 'r') as f:
    label_names = [line.strip() for line in f.readlines()]

## MediaPipe inicjalizacja

In [5]:
mp_holistic = mp.solutions.holistic
holistic = mp_holistic.Holistic(static_image_mode=False, model_complexity=1)
mp_drawing = mp.solutions.drawing_utils

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


# Bufor sekwencji

In [6]:
def normalize_sequence(sequence_array):
    sequence_array = np.array(sequence_array)
    origin = sequence_array[0][0]  # Nadgarstek 1. klatki
    return sequence_array - origin

## Ekstrakcja 21 punktów prawej dłoni

In [7]:
def extract_hand_landmarks(results):
    if results.right_hand_landmarks:
        return np.array([[lm.x, lm.y, lm.z] for lm in results.right_hand_landmarks.landmark])
    return None

In [20]:
import time
from pathlib import Path

# --- Parametry ---
SEQ_LENGTH = 30
THRESHOLD = 0.8
COOLDOWN_SECONDS = 1.0
VIDEO_PATH = "video.mp4"
OUTPUT_FOLDER = Path("results")
OUTPUT_FOLDER.mkdir(exist_ok=True)

# Pętla po modelach

# --- Główna pętla testu ---
cap = cv2.VideoCapture(VIDEO_PATH)
frame_count = 0
all_results = {}
for model_name, model in models.items():
    cap.set(cv2.CAP_PROP_POS_FRAMES, 0)  # restart video
    sequence = []
    recognized_text = ""
    last_prediction_time = 0

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # Ekstrakcja landmarków
        hand_landmarks = extract_hand_landmarks(holistic.process(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)))

        if hand_landmarks is not None:
            sequence.append(hand_landmarks)

            if len(sequence) == SEQ_LENGTH:
                # Normalizacja i predykcja
                norm_seq = normalize_sequence(sequence)
                input_seq = norm_seq.reshape(1, SEQ_LENGTH, 63)
                pred = model.predict(input_seq, verbose=0)[0]

                max_prob = np.max(pred)
                label = label_names[np.argmax(pred)]

                # Sprawdzenie progu i cooldownu
                current_time = time.time()
                cooldown_remaining = max(0, COOLDOWN_SECONDS - (current_time - last_prediction_time))
                if max_prob > THRESHOLD and cooldown_remaining == 0:
                    recognized_text += label
                    last_prediction_time = current_time

                sequence.pop(0)  # przesuwamy okno

        else:
            sequence = []  # brak ręki → restart sekwencji

        # Zapis wyników do pliku tekstowego
    with open(f"results_{model_name}.txt", "w") as f:
        f.write(recognized_text)

    all_results[model_name] = recognized_text
    del model  # zwolnienie pamięci

    print(f"Model {model_name} zakończył. Tekst: {recognized_text}")
    cap.set(cv2.CAP_PROP_POS_FRAMES, 0)  # reset dla kolejnego modelu

cap.release()

Model lstm_2025_08_18 zakończył. Tekst: AAAA_AA_AA_AABBBBSZC_CZ_ZCC_CC_CC_CCZCHCHCHCHCHCHDDPE_EE_EPE_EE_EE_EE_EE_EFFFFFTBGGGSZRHHHAJJAJJJRKKDL_LL_LL_LL_LL_LN_NN_NNNNNNNN_NN_NN_NOO_OOO_OO_OO_OPPPPRRRRRRRZSSSSS_SS_SO_OBBSZSZSZUUUUWWWWWYYYYYZ_ZC_CZ_XZ_ZZ_Z


In [21]:
import glob

# Wczytanie wyników z plików
all_results = {}
for filepath in glob.glob("results_*.txt"):
    model_name = filepath.split("_")[1].split(".")[0]  # np. "lstm" z "results_lstm.txt"
    with open(filepath, "r") as f:
        all_results[model_name] = f.read().strip()

# Porównanie zgodności modeli
model_names = list(all_results.keys())
for i in range(len(model_names)):
    for j in range(i+1, len(model_names)):
        m1, m2 = model_names[i], model_names[j]
        seq1, seq2 = all_results[m1], all_results[m2]
        min_len = min(len(seq1), len(seq2))
        agreement = sum([1 for k in range(min_len) if seq1[k]==seq2[k]]) / min_len
        print(f"Zgodność {m1} vs {m2}: {agreement*100:.2f}%")


Zgodność cnn vs lstm: 32.86%
